In [ ]:
# Import 1
# in your notebook cell
import sys
# path relative to your notebook
sys.path.append("/Users/joel/Desktop/daimil10/capstone_1/DC-Metro-Crime-Data-2007---2017/src")
# import as usual
import functions

# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# Import 2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style

In [ ]:
# Read file in and drop first two rows
# First two rows seem to make like no sense with the rest of this data set
df = functions.read_file('/Users/joel/Desktop/daimil10/capstone_1/DC-Metro-Crime-Data-2007---2017/data/dc_crime_add_vars.csv')
df = df.drop(['Unnamed: 0', 'X'], axis=1)
df.head()

In [ ]:
# Describing the Dataset 1
df.describe()

In [ ]:
# Describing the Dataset 2
df.info()

In [ ]:
# Describing the Dataset 3
df.index

In [ ]:
# Describing the Dataset 4
df.columns

In [ ]:
# Describing the Dataset 5
missing_v = df.isnull().sum()
missing_v

# new dataframe of al missing values
data = {
    'DISTRICT': [200],
    'PSA': [251],
    'NEIGHBORHOOD_CLUSTER': [4705],
    'BLOCK_GROUP': [1091],
    'CENSUS_TRACT': [1091],
    'VOTING_PRECINCT': [84],
    'START_DATE': [13],
    'END_DATE': [11651],
    'XBLOCK': [0],
    'YBLOCK': [0],
    'date': [0],
    'year': [0],
    'month': [0],
    'day': [0],
    'hour': [0],
    'minute': [0],
    'second': [0]
}

# show the missing data
df_bar = pd.DataFrame(data)
plt.figure(figsize=(12, 6))
df_bar.plot(kind='bar', edgecolor='black')

plt.xlabel('Col Name')
plt.ylabel('Missing Values')
plt.title('Missing Values in Dataset')
plt.show()

In [ ]:
# Describing the Dataset 6
# Went from 342867 to 325340 = 17527 removed which is roughly 5% of the dataset 
df = functions.drop_rows_with_missing_data(df)
df.info()

In [ ]:
# Describing the Dataset 7
functions.describe_dataframe(df)

In [ ]:
# Describing the Dataset 8
functions.get_column_features(df)

In [ ]:
# Can use this is need to or can slice the other df accordingly
 
# Slice df to only include the offense of homicide

# Convert datetimes

# Make new start and end hour column
df_2 = df[df['OFFENSE']=='HOMICIDE']
df_2['START_DATE'] = pd.to_datetime(df_2['START_DATE'])
df_2['END_DATE'] = pd.to_datetime(df_2['END_DATE'])
df_2['start_hour'] = df_2['START_DATE'].dt.hour
df_2['end_hour'] = df_2['START_DATE'].dt.hour
df_2 = df_2.drop('hour', axis=1)
df_2.head()

In [ ]:
# Can use this is need to or can slice the other df accordingly 

# 1100 rows now
hom_df = df_2

# Slicing df one last time to get only my desired columns
hom_df = hom_df[['REPORT_DAT',
                'SHIFT',
                'OFFENSE',
                'METHOD',
                'BLOCK',
                'DISTRICT',
                'PSA',
                'WARD',
                'ANC',
                'NEIGHBORHOOD_CLUSTER',
                'BLOCK_GROUP',
                'VOTING_PRECINCT',
                'START_DATE',
                'END_DATE',
                'XBLOCK',
                'YBLOCK',
                'date',
                'year',
                'start_hour',
                'end_hour',
                'crimetype']]

#Show that it worked
hom_df.columns

In [ ]:
# Have homicides increased over this decade?
# Line Chart

from matplotlib import style

homicides_df = df[df['OFFENSE'] == 'HOMICIDE']
# Group the data by year and count the number of homicides per year
homicides_by_year = homicides_df.groupby('year').size()

plotter = functions.Plot()
plotter.line_chart(homicides_by_year, 'Year', 'Homicides', 'Number of Homicides by Year: 2008 - 2017')

In [ ]:
# Which wards / neighborhoods had the highest number of homicides?
# Bar Chart

# Filter the DataFrame for homicides
homicides_df = df[df['OFFENSE'] == 'HOMICIDE']

# Get homicides by district value counts and sort values
homicides_by_location = homicides_df['WARD'].value_counts().sort_values()

plotter.bar_chart(homicides_by_location, 'Ward', 'Homicides', 'Number of Homicides by Ward')

In [ ]:
# Is there any correlation between the time of the day and the occurrence of homicides?
# Heat Map

# Extract the hour from the 'REPORT_DAT' column
df['Time_of_Day'] = pd.to_datetime(df['REPORT_DAT']).dt.hour

# Create a binary(dummy) column for the cat columns
df['Homicide'] = df['OFFENSE'].apply(lambda x: 1 if 'HOMICIDE' in x else 0)
df['Shift_Evening'] = df['SHIFT'].apply(lambda x: 1 if 'EVENING' in x else 0)
df['Shift_Midnight'] = df['SHIFT'].apply(lambda x: 1 if 'MIDNIGHT' in x else 0)

# Select the columns of interest
correlation_df= df[['Time_of_Day', 'Homicide', 'Shift_Evening', 'Shift_Midnight']]
corr_matrix = correlation_df.corr()

plotter.heatmap(corr_matrix, 'Matrix Columns', 'Matrix Column', 'Correlation Heatmap: Homicide & Time of Day')

In [ ]:
# Are homicides more likely to occur with a gun or a knife? 
# Stacked Plot

#replace method
df['OFFENSE'] = df['OFFENSE'].replace('ASSAULT W/DANGEROUS WEAPON', 'ASSAULT W/ WEAPON')

# Select list of violent offenses
offenses = ['HOMICIDE', 'ROBBERY', 'ASSAULT W/ WEAPON', 'BURGLARY']

# Filter the DataFrame using isin cause i keep getting value error? 
filtered_df = df[df['OFFENSE'].isin(offenses)]

# Group the data by offense and method, and count
# .size()is used after grouping the data to count the number of occurrences 
# for each group. It returns a Series or DataFrame with the counts.
# .unstack() is applied to a hierarchical index (such as the result of a groupby operation) 
# and reshapes the data by "unstacking" one level of the index to create columns.
grouped_data = filtered_df.groupby(['OFFENSE', 'METHOD']).size().unstack()

# Calculate the proportions for each method within each offense
grouped_data_proportions = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100  # Multiply by 100 to get percentages

plotter.stacked(grouped_data_proportions, 'Offense', 'Proportion(%)', 'Proportion of Offenses by Method: 2007 - 2017')

In [ ]:
# Extra: Folium Experiment?


import folium
from folium.plugins import HeatMap

# Filter the DataFrame to include only homicides
homicides_df = df[df['OFFENSE'] == 'HOMICIDE']

# Group the data by ward and count the number of homicides in each ward
PSA_homicide_counts = homicides_df['WARD'].value_counts()

# Find the ward with the highest amount of homicides
PSA_with_highest_homicides = PSA_homicide_counts.idxmax()

# Filter the DataFrame to include only the data for the ward with the highest amount of homicides
PSA_homicides_df = homicides_df[homicides_df['WARD'] == PSA_with_highest_homicides]

# Create a map centered on Washington D.C.
map_homicides = folium.Map(location=[38.9072, -77.0369], zoom_start=12)

# Generate coordinates for heatmap from the filtered DataFrame
heat_data = PSA_homicides_df[['YBLOCK', 'XBLOCK']].values

#for legend
num_66 = homicides_df['WARD'].value_counts()[8]

legend_html = f'''
                <div style="position: fixed; 
                            bottom: 50px; left: 50px; width: 120px; height: 60px; 
                            background-color: rgba(255, 255, 255, 0.9); z-index:9999; 
                            font-size:14px;border-radius: 5px;
                            ">
                <strong>WARD 8</strong><br>
                Total Homicides: {num_66}
                </div>
                '''

legend_div = folium.features.DivIcon(
    icon_size=(120, 80),
    icon_anchor=(10, 10),
    html=legend_html
)

folium.Marker(
    location=[38.8457, -77.0097],  # Adjust the location as per your preference
    icon=legend_div,
).add_to(map_homicides)

# Add the heatmap layer to the map
HeatMap(heat_data).add_to(map_homicides)

# Display the map
map_homicides


In [ ]:
num_66 = homicides_df['PSA'].value_counts()[604]